### Sqaure root decomposition(평방분할)
- 값 $N$ 개를 $\Omicron(\sqrt N)$ 개의 연속한 구간들로 나누어서 관리하는 방법
- 각 구간에 속한 값들도 $\Omicron(\sqrt N)$ 개로 나눈다. 즉, (구간 수) * (구간 내 원소 수) = $\sqrt N * \sqrt N = N$ 이다.
- N이 제곱수가 아닌 경우 적당히 올림/내림해서 쓴다.

### Mo's algorithm
- 평방분할을 구간 쿼리에 적용하는 방법
- 배열의 업데이트가 없고, 구간 쿼리만 존재하는 경우에 사용할 수 있다.
- 쿼리를 정렬한 뒤, 쿼리의 구간을 옮겨가며 쿼리를 처리한다.
  - 쿼리를 순서대로 처리하지 않기 때문에 오프라인 쿼리이다.
- 구간 쿼리의 처리시간을 $T(N)$, 쿼리의 개수가 $M$ 일때 $\Omicron(M + T(N) * N \sqrt N)$ 의 시간복잡도를 가진다.

#### 11659 구간 합 구하기 4
- Mo's는 구조가 여기서 크게 달라지지 않는다.

In [ ]:
import io, os, math, sys
input=io.BytesIO(os.read(0,os.fstat(0).st_size)).readline

def sol() :
  N, M = map(int, input().split())
  L = [*map(int, input().split())]

  sqrtN = math.isqrt(N)
  Q = []
  for i in range(M):
    s, e = map(int, input().split())
    Q.append((s-1, e, i))
  Q.sort(key=lambda x: (x[0] // sqrtN, x[1])) #쿼리를 평방분할한다.

  #첫번째 쿼리를 직접 구한다
  acc = 0
  res = [0] * M
  s, e = Q[0][0], Q[0][1]
  for i in range(s, e):
    acc += L[i]
  res[Q[0][2]] = acc

  #바로 이전 쿼리의 결과를 이용해서 새로운 쿼리를 구한다.
  for ns, ne, idx in Q[1:]:
    while ns < s: #이전쿼리보다 s가 작으면 왼쪽으로 구간을 확장하면서 더한다.
      s -= 1
      acc += L[s]
    
    while e < ne: #이전쿼리보다 e가 크면 오른쪽으로 구간을 확장하면서 더한다.
      acc += L[e]
      e += 1
    
    while ns > s: #이전쿼리보다 s가 크면 왼쪽으로 구간을 축소하면서 뺀다.
      acc -= L[s]
      s += 1
    
    while e > ne: #이전쿼리보다 e가 작으면 오른쪽으로 구간을 축소하면서 뺀다.
      e -= 1
      acc -= L[e]
    
    res[idx] = acc #쿼리의 결과를 저장한다.
  
  sys.stdout.write('\n'.join(map(str, res)))

sol()

- 쿼리를 정렬하는데에 $\Omicron(M \log M)$ 의 시간이 소요된다.
- 이전 쿼리로부터 영역을 확장해야 하므로, 첫번째 쿼리는 직접 구해준다.
  - $\Omicron(N)$
- 구간 쿼리의 처리 시간은 슬라이딩 윈도우로 구할 수 있다.
  - $T(N) = \Omicron(1)$
- 이제 각 평방분할이 어떻게 되는지 살펴보자.
  - 어떤 쿼리를 $\sqrt N$ 개의 블럭으로 평방분할 했을 때, 블럭 내에있는 $\sqrt N$ 개의 쿼리의 구간 $Q_i = [(s_1, e_1), (s_2, e_2), ... (s_n, e_n)]$ 에 대하여,
  1. 이전 쿼리와 같은 블록($\frac {s}{\sqrt N}$) 에 속하는 경우
      - 이전 쿼리와 $s$ 의 차이는 최대 $\sqrt N$ 이다. 따라서 총합 $\Omicron(N \sqrt N)$ 만큼 $s$ 가 변화한다.
      - 같은 블럭에 속해있는 한, $e$ 는 증가하는 순으로만 정렬돼있으므로, 같은 블럭내의 최댓값 $\max(Q.e)$ 에서 최솟값 $\min(Q.e)$ 의 차이만큼만 움직이게 된다. 즉 단조 증가한다.
        - 한 블럭 $Q$ 에서 $e$ 는 최대 $N$ 만큼만 변화한다.
      - 이러한 블럭이 $\Omicron(\sqrt N)$ 개 있으므로 총합 $\Omicron(N \sqrt N)$ 만큼 $e$ 가 변화한다.
  2. 이전 쿼리와 블럭이 다른 경우
      - 이 경우 $s$ 는 최대 $N$ 만큼 변화한다.
      - 쿼리를 $\sqrt N$ 개의 블럭으로 분할했으므로, 이런 경우는 최대 $\Omicron(\sqrt N)$ 번밖에 일어나지 않는다.
        - 서로다른 블럭의 index 차이가 $\Omicron(\sqrt N)$ 이므로, 전체 총합은 $\Omicron(N \sqrt N)$ 이다.
      - $e$ 는 $\Omicron(N)$ 만큼 바뀌고, 이 경우는 $\Omicron(\sqrt N)$ 번 일어난다.
        - 전체 총합은 $\Omicron(N \sqrt N)$ 이다.
  - $s, e$ 의 변화량의 총합은 $\Omicron(N \sqrt N)$ 이다.
    - 따라서 쿼리를 전부 처리하는데 걸리는 시간은 $\Omicron(M + N \sqrt N)$ 이다.
- 전체 시간복잡도는 $\Omicron(M \log M + N \sqrt N)$ 이다.

### 문제 유형
- 구분되는 수의 개수를 세기
- 정확히 k번 등장하는 수의 개수를 세기